### Introduction to machine learning with TensorFlow

This notebook is intended to be run in a local Python environment with access to the TensorFlow package.

In [1]:
from modules.tensorflow_functions import *
from ucimlrepo import fetch_ucirepo

Read in data - we are using the abalone mollusc dataset from the UCI ML dataset repository:

In [2]:
abalones = fetch_ucirepo(id=1)

abalones_df = abalones.data.original

abalones_df.head(5)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Using the UCI ML repo package, the dataset comes with the variables already split into features and labels:

In [3]:
# Features:
abalones_x = abalones.data.features

# Labels:
abalones_y = abalones.data.targets

print("The features of this dataset are:\n")
for col in abalones_x.columns: print(col)

print("\nThe labels of this dataset are:\n")
for col in abalones_y.columns: print(col)

The features of this dataset are:

Sex
Length
Diameter
Height
Whole_weight
Shucked_weight
Viscera_weight
Shell_weight

The labels of this dataset are:

Rings


So the machine learning task is to use some combination of the sex, length, diameter, height and weights of the abalone molluscs to predict the number of shell rings.

We can see our feature set includes the variable `Sex`, which is a categorical variable. We will recode this into dummy variables:

In [4]:
encoded_sex = pd.get_dummies(abalones_df["Sex"], prefix = "Sex", dtype = int)

abalones_df = pd.concat([abalones_df, encoded_sex], axis = 1)

abalones_df.head(5)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


We can drop the original `Sex` variable from the feature set now that we have replaced it with numeric dummy variables:

In [5]:
abalones_df = abalones_df.drop("Sex", axis = 1)

We will use functions defined in [tensorflow_functions.py](modules/tensorflow_functions.py) to build and train a linear regression model on the abalone DataFrame.

The machine learning model will depend on three hyperparameters:

- [Learning rate](https://developers.google.com/machine-learning/glossary/#learning-rate)
- [Epochs](https://developers.google.com/machine-learning/glossary/#epoch)
- [Batch size](https://developers.google.com/machine-learning/glossary/#batch_size)

It will also depend on the features we select to predict the label, `Rings`.

In [6]:
# Define hyperparameters
model_learning_rate = 0.1
model_epochs = 20
model_batch_size = 100

# Set label and features
model_label = "Rings"
model_features = ["Length", "Diameter", "Height"]

# Build model
base_model = build_lr_model(learning_rate = model_learning_rate)

# Train model
weight, bias, epochs, rmse = train_model(
    model =  base_model,
    df = abalones_df,
    features = model_features,
    label = model_label,
    epochs = model_epochs,
    batch_size = model_batch_size
)

# Plot the model
plot_the_model(weight, bias, model_features, model_label)
plot_the_loss_curve(epochs, rmse)

AttributeError: module 'keras.api._v2.keras.optimizers.experimental' has no attribute 'RSprop'